In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB
import json

In [2]:
# Define the data folder path
folder_path = './data/question_1b'
file_type = ".json"

with open(f'{folder_path}/appliance_params{file_type}', 'r') as f:
    appliance_params = json.load(f)

with open(f'{folder_path}/bus_params{file_type}', 'r') as f:
    bus_params = json.load(f)

with open(f'{folder_path}/consumer_params{file_type}', 'r') as f:
    consumer_params = json.load(f)

with open(f'{folder_path}/DER_production{file_type}', 'r') as f:
    DER_production = json.load(f)

with open(f'{folder_path}/usage_preferences{file_type}', 'r') as f:
    usage_preference = json.load(f)


In [3]:
#System Parameters
bus_data = bus_params[0]
tau_imp = bus_data['import_tariff_DKK/kWh']
tau_exp = bus_data['export_tariff_DKK/kWh']
max_import = bus_data['max_import_kW']
max_export = bus_data['max_export_kW']
electricity_prices = bus_data['energy_price_DKK_per_kWh']

In [4]:
#PV parameters
pv_data = appliance_params['DER'][0]
pv_max_power = pv_data['max_power_kW']
pv_profile = DER_production[0]['hourly_profile_ratio']
pv_prod_hourly = [pv_max_power * ratio for ratio in pv_profile] # PV production per hour (kW) (No curtailment)

In [5]:
#Load parameters
load_data = appliance_params['load'][0]
L_max = load_data['max_load_kWh_per_hour'] # Maximum power consumption (kW)
L_ref_ratio = usage_preference[0]['load_preferences'][0]['hourly_profile_ratio'] # Minimum daily consumption (kWh)
L_ref = [L_max * ratio for ratio in L_ref_ratio] # PV production per hour (kW) (No curtailment)

In [6]:
print(L_ref)

[0.165, 0.12, 0.12, 0.12, 0.22499999999999998, 1.44, 2.2800000000000002, 2.4000000000000004, 1.8900000000000001, 0.66, 0.75, 1.0499999999999998, 0.8999999999999999, 0.8400000000000001, 1.35, 1.9500000000000002, 2.34, 2.7, 2.94, 2.64, 0.22499999999999998, 0.44999999999999996, 0.22499999999999998, 0.165]


In [7]:
#sum of L_ref
print(sum(L_ref))
print(sum(pv_prod_hourly))
print(sum(pv_profile))

27.945000000000004
13.469999999999999
4.49


In [8]:
#Temporal parameters
T = len(electricity_prices)  # 24 hours
Times = range(T) # Time horizon (24 hours)
# Discomfort parameter
alpha = 17

In [9]:
model = gp.Model("Energy_Optimization")

# Decision variables:
L_t = model.addVars(Times, lb=0, name="L_t")  # Load consumption (kW)
C_t = model.addVars(Times, lb=0, name="C_t")  # Energy curtailed from PV (kW)
G_imp_t = model.addVars(Times, lb=0, ub=max_import, name="G_imp_t")  # Grid import (kW)
G_exp_t = model.addVars(Times, lb=0, ub=max_export, name="G_exp_t")  # Grid export (kW)
D_t = model.addVars(Times, lb=0, name="D_t")  # Discomfort (kW)


Set parameter Username
Set parameter LicenseID to value 2617496
Academic license - for non-commercial use only - expires 2026-02-03


In [10]:
#Objective function
model.setObjective(
           gp.quicksum(G_imp_t[t] * (tau_imp + electricity_prices[t] + 
                    G_exp_t[t]) * (tau_exp - electricity_prices[t]) for t in Times) + 
                    alpha * gp.quicksum(D_t[t] for t in Times),
    GRB.MINIMIZE
)

In [11]:
#add constraints

Curtailment_constraint = [
	model.addLConstr(C_t[t] <= pv_prod_hourly[t], name=f"Curtailment")
	for t in Times
]

power_balance_constraint = [
	model.addLConstr(G_imp_t[t] - G_exp_t[t] == L_t[t] - pv_prod_hourly[t] + C_t[t], name="power_balance")
	for t in Times
]

discomfort_constraint_1 = [
	model.addLConstr(D_t[t] >= L_t[t] - L_ref[t], name="discomfort_1")
	for t in Times
]
discomfort_constraint_2 = [
	model.addLConstr(D_t[t] >= -(L_t[t] - L_ref[t]), name="discomfort_2")
	for t in Times
]

In [12]:
#solve optimization problem
model.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 6800HS Creator Edition, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 96 rows, 120 columns and 216 nonzeros
Model fingerprint: 0x3e878fee
Model has 24 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 2e+01]
  QObjective range [3e+00, 6e+00]
  Bounds range     [5e+02, 1e+03]
  RHS range        [1e-01, 3e+00]
Presolve removed 24 rows and 9 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 93 rows and 115 columns
Presolve time: 0.03s
Presolved: 5 rows, 7 columns, 12 nonzeros
Presolved model has 1 bilinear constraint(s)
Variable types: 7 continuous, 0 integer (0 binary)
Found heuristic solution: objective 279.8637000

Root relaxation: cutoff, 0 iterations, 0.00 seconds (0.00 work units)

Explored 1 nodes (0 simp

In [13]:
#print results
if model.status == GRB.OPTIMAL:
    print("\nOptimal solution found:")
    print(f"Total cost: {model.objVal:.2f} DKK")
    print("Hour | Load (kW) | Curtailment (kW) | Grid Import (kW) | Grid Export (kW) | PV Production (kW) | Electricity Price (DKK/kWh)")
    for t in Times:
        print(f"{t:4d} | {L_t[t].X:9.2f} | {C_t[t].X:15.2f} | {G_imp_t[t].X:15.2f} | {G_exp_t[t].X:15.2f} | {pv_prod_hourly[t]:18.2f} | {electricity_prices[t]:25.2f}")


Optimal solution found:
Total cost: 279.86 DKK
Hour | Load (kW) | Curtailment (kW) | Grid Import (kW) | Grid Export (kW) | PV Production (kW) | Electricity Price (DKK/kWh)
   0 |      0.00 |            0.00 |            0.00 |            0.00 |               0.00 |                      1.10
   1 |      0.00 |            0.00 |            0.00 |            0.00 |               0.00 |                      1.05
   2 |      0.00 |            0.00 |            0.00 |            0.00 |               0.00 |                      1.00
   3 |      0.00 |            0.00 |            0.00 |            0.00 |               0.00 |                      0.90
   4 |      0.00 |            0.00 |            0.00 |            0.00 |               0.00 |                      0.85
   5 |      0.15 |            0.00 |            0.00 |            0.00 |               0.15 |                      1.01
   6 |      0.42 |            0.00 |            0.00 |            0.00 |               0.42 |              

In [14]:
print(sum(L_ref))

27.945000000000004


In [15]:
print(sum(pv_prod_hourly))

13.469999999999999


In [16]:
#print import
print(sum([G_imp_t[t].X for t in Times]))

9.389999999784742


In [17]:
#sum L_t
print(f"\nTotal Load Consumption: {sum(L_t[t].X for t in Times):.2f} kWh")


Total Load Consumption: 19.50 kWh
